## IMPORTING THE REQUIRED LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import math
import os
import requests
import json
import re

## READING THE CSV FILE INTO A DATAFRAME

In [2]:
df = pd.read_csv('Met_unclean_data.csv')

In [3]:
df.head(3)

,Object Number,Object ID,Department,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,...,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository,Tags
0,62.16,1674,American Decorative Arts,Side Chair,Side Chair,American,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Furniture,NaN,http://www.metmuseum.org/art/collection/search...,5/13/2019 8:00,"Metropolitan Museum of Art, New York, NY",Chairs
1,13.7.17,2430,American Decorative Arts,Pitcher,Covered Pitcher,American,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Ceramics,NaN,http://www.metmuseum.org/art/collection/search...,5/13/2019 8:00,"Metropolitan Museum of Art, New York, NY",Pitchers
2,33.85.11,6406,American Decorative Arts,Plate,Plate,Dutch,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Ceramics,NaN,http://www.metmuseum.org/art/collection/search...,5/13/2019 8:00,"Metropolitan Museum of Art, New York, NY",NaN


In [4]:
df.shape

(204, 41)

## SUBSETTING THE DATAFRAME TO INCLUDE ONLY THE REQUIRED COLUMNS

In [5]:
req_data = df[['Object ID', 'Department', 'Object Name', 'Title', 'Culture', 'Artist Nationality', 'Object Begin Date', 'Object End Date',
                'Medium', 'Credit Line', 'Country']]

In [6]:
req_data.head(5)

,Object ID,Department,Object Name,Title,Culture,Artist Nationality,Object Begin Date,Object End Date,Medium,Credit Line,Country
0,1674,American Decorative Arts,Side Chair,Side Chair,American,NaN,1794,1799.0,"Mahogany, ebony, ash, birch, white pine","Friends of the American Wing Fund, 1962",United States
1,2430,American Decorative Arts,Pitcher,Covered Pitcher,American,NaN,1700,1900.0,Earthenware,"Rogers Fund, 1913",United States
2,6406,American Decorative Arts,Plate,Plate,Dutch,NaN,1740,1760.0,Earthenware,"Rogers Fund, 1933",Netherlands
3,7616,American Decorative Arts,Soup bowl,Soup Bowl,American,NaN,1885,1885.0,Porcelain,"Gift of Mr. and Mrs. Franklin Chace, 1969",United States
4,15012,American Decorative Arts,"Painting, miniature",Portrait of a Lady,American,NaN,1840,1840.0,Watercolor on ivory,"Fletcher Fund, 2006",NaN


In [7]:
req_data.shape

(204, 11)

## CHECK THE NUMBER OF MISSING VALUES IN EACH COLUMN

In [8]:
req_data.isna().sum()

Object ID               0
Department              1
Object Name             1
Title                  10
Culture               120
Artist Nationality    117
Object Begin Date       0
Object End Date         2
Medium                  2
Credit Line             0
Country               172
dtype: int64

## START WITH THE 'Department' COLUMN

<I> First, we check the unique values present in the column. If any discrepancies - such as non standard form or leading/trailing spaces are found, we fix them. Later, we check the missing value and note how we can deal with it </I>

In [9]:
req_data['Department'].unique()

array(['American Decorative Arts', 'Arms and Armor', 'Asian Art',
       'Costume Institute', 'European Sculpture and Decorative Arts',
       'Greek and Roman Art', 'Photographs',
       'Arts of Africa, Oceania, and the Americas', nan,
       'Ancient Near Eastern Art', 'Drawings and Prints', 'Islamic Art',
       'Robert Lehman Collection', 'Medieval Art', 'The Cloisters',
       'Modern and Contemporary Art', 'Egyptian Art'], dtype=object)

<I> next, we take a look at the row with the missing Dept value. The original dataframe also contains a link to the object's museum page. We can navigate to this page and identiy the Department's name to impute this missing value </I>

In [10]:
df.loc[76]

Object Number                                                    1994.35.185
Object ID                                                             317005
Department                                                               NaN
Object Name                                                            Cloth
Title                                                         Carrying Cloth
Culture                                                              Quechua
Period                                                                   NaN
Dynasty                                                                  NaN
Reign                                                                    NaN
Portfolio                                                                NaN
Artist Role                                                              NaN
Artist Prefix                                                            NaN
Artist Display Name                                                      NaN

In [11]:
# Search for the object in the museum using the Object Number - Also known as the Accession Number on the museum's page

In [12]:
df.loc[76]['Object Number']

'1994.35.185'

<I> Having identified the Department, we plug it into our data </I>

In [13]:
req_data['Department'][76] = 'The Michael C. Rockefeller Wing'

<ipython-input-13-219b34d17d30>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Department'][76] = 'The Michael C. Rockefeller Wing'
C:\Users\sidew\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
req_data['Department'].unique()

array(['American Decorative Arts', 'Arms and Armor', 'Asian Art',
       'Costume Institute', 'European Sculpture and Decorative Arts',
       'Greek and Roman Art', 'Photographs',
       'Arts of Africa, Oceania, and the Americas',
       'The Michael C. Rockefeller Wing', 'Ancient Near Eastern Art',
       'Drawings and Prints', 'Islamic Art', 'Robert Lehman Collection',
       'Medieval Art', 'The Cloisters', 'Modern and Contemporary Art',
       'Egyptian Art'], dtype=object)

<I> Since we can't be sure of the entries, its better to trim any unwanted spaces for all entries, if they exist. We can use the <B>strip()</B> function for this </I>

In [15]:
for i in range(req_data.shape[0]):
    req_data['Department'][i] = req_data['Department'][i].strip()

<ipython-input-15-8602ba4dfd78>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Department'][i] = req_data['Department'][i].strip()


In [16]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             1
Title                  10
Culture               120
Artist Nationality    117
Object Begin Date       0
Object End Date         2
Medium                  2
Credit Line             0
Country               172
dtype: int64

## 'Object Name' COLUMN'S MISSING VALUE

In [17]:
df.loc[165]

Object Number                                                 SL.6.2016.88.1
Object ID                                                             656878
Department                                       Modern and Contemporary Art
Object Name                                                              NaN
Title                      New York City 2[unfinished, formerly New York ...
Culture                                                                  NaN
Period                                                                   NaN
Dynasty                                                                  NaN
Reign                                                                    NaN
Portfolio                                                                NaN
Artist Role                                                           Artist
Artist Prefix                                                            NaN
Artist Display Name                                            Piet Mondrian

<I> we adopt a similar approach of navigating to the page in the museum's website. </I> <B>However, no such record seems to exist in the museum's page. </B> <I>So we delete this record which contains false/incorrect/outdated information</I>

In [18]:
df.loc[165]['Object Number']

'SL.6.2016.88.1'

#### Moving forward, the strip function will be applied to all the columns containing string type values

In [19]:
for i in range(req_data.shape[0]):
    if not req_data['Object Name'][i] != req_data['Object Name'][i]:
        req_data['Object Name'][i] = req_data['Object Name'][i].strip()

<ipython-input-19-8b5295514134>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Object Name'][i] = req_data['Object Name'][i].strip()


### DROP THE RECORD BEARING INDEX NUMBER 165 FROM OUR SUBSET DATAFRAME

In [20]:
req_data = req_data.drop(165)
req_data = req_data.reset_index(drop = True)

### WE ALSO NOTICE THAT THE VALUES ARE IN NON STANDARDISED FORM, SO WE FIX THEM

In [21]:
req_data['Object Name'].unique()

array(['Side Chair', 'Pitcher', 'Plate', 'Soup bowl',
       'Painting, miniature', 'Arrowhead (Yanonē)',
       'Knife handle (Kozuka)', 'Coin (Thaler)', 'Two drawings',
       'Partisan', 'Sword guard (Tsuba)', 'Dirk with sheath',
       'Hanging scroll mounted as handscroll', 'Girdle ornament',
       'Bracelet', 'Print', 'Album leaf', 'Inrō', 'Basket', 'Piece',
       'Hanging scroll', 'Drawing', 'Belt', 'Pumps', 'Evening dress',
       'Shoes', 'Earrings', 'Cap', 'Hat', 'Mules', 'Gloves', 'Ensemble',
       'Necklace', 'Jacket', 'Chemisette', 'Fan', 'Undersleeves',
       'Evening shoes', 'Knife and fork with case', 'Swags',
       'Jug with cover', 'Plaque', 'Snuffbox', 'Chess set', 'Toy',
       'Souvenir spoon', 'Border', 'Fragment', 'Borders', 'Panel',
       'Statuette of a seated goddess', 'Axehead',
       'Fibula, boat-shaped type', 'Statuette of a woman, seated', 'Seal',
       'Photograph', 'Glass positive', 'Negative', 'Horn', 'Snake figure',
       'Pendant', 'Painting

The value <B>'Drawing Ornament & Architecture' </B>and <B>'Drawing ; Ornament and Architecture' </B>represent the same entry level. However, they have been represented differently.

* <U>We fix such cases across the column's values</U>.
 
* Note that some values, though they seem similar, have not been modified since we have no clear idea of what the object actually is.

In [22]:
req_data[(req_data['Object Name'] == 'Drawing Ornament & Architecture') | 
         (req_data['Object Name'] == 'Drawing ; Ornament and architecture')]

,Object ID,Department,Object Name,Title,Culture,Artist Nationality,Object Begin Date,Object End Date,Medium,Credit Line,Country
99,367617,Drawings and Prints,Drawing Ornament & Architecture,Francis W. Little House: Furniture Plan,NaN,American,1908,1914.0,Graphite and orange and brown pencil,"Purchase, Emily Crane Chadborne Bequest, 1972",NaN
104,386510,Drawings and Prints,Drawing ; Ornament and architecture,"Ceiling Design, Half of a Circle",NaN,British,1800,1899.0,Watercolor with gold,Gift of the Royal Institute of British Archite...,NaN


In [23]:
req_data['Object Name'][99] = 'Drawing Ornament & Architecture'
req_data['Object Name'][104] = 'Drawing Ornament & Architecture'

<ipython-input-23-30f415174fb3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Object Name'][99] = 'Drawing Ornament & Architecture'
<ipython-input-23-30f415174fb3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Object Name'][104] = 'Drawing Ornament & Architecture'


Standardising the <B>Bowl</B> entries

In [24]:
req_data[(req_data['Object Name'] == 'Soup bowl') | (req_data['Object Name'] == 'Bowl')]

,Object ID,Department,Object Name,Title,Culture,Artist Nationality,Object Begin Date,Object End Date,Medium,Credit Line,Country
3,7616,American Decorative Arts,Soup bowl,Soup Bowl,American,NaN,1885,1885.0,Porcelain,"Gift of Mr. and Mrs. Franklin Chace, 1969",United States
138,486599,Modern and Contemporary Art,Bowl,"""Tard la Nuit"" Bowl",NaN,Italian,1989,1989.0,Glass,"Gift of Peter T. Joseph, 1995",NaN
139,486599,Modern and Contemporary Art,Bowl,"""Tard la Nuit"" Bowl",NaN,Italian,1989,1989.0,Glass,"Gift of Peter T. Joseph, 1995",NaN


In [25]:
req_data['Object Name'][3] = 'Bowl'
req_data['Object Name'][138] = 'Bowl'
req_data['Object Name'][139] = 'Bowl'

<ipython-input-25-71c497cb3dda>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Object Name'][3] = 'Bowl'
<ipython-input-25-71c497cb3dda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Object Name'][138] = 'Bowl'
<ipython-input-25-71c497cb3dda>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Object Name'][139] = 'Bowl'


Standardising the <B>Baseball Card</B> entries

In [26]:
req_data[(req_data['Object Name'] == 'Baseball card') | (req_data['Object Name'] == 'Baseball card, print') |
         (req_data['Object Name'] == 'Print, baseball card') | (req_data['Object Name'] == 'Print, baseball')]

,Object ID,Department,Object Name,Title,Culture,Artist Nationality,Object Begin Date,Object End Date,Medium,Credit Line,Country
111,400013,Drawings and Prints,Baseball card,"Sullivan, Left Field, Chicago, from the Gold C...",NaN,NaN,1887,1887.0,Commercial color lithography reproducing drawings,"The Jefferson R. Burdick Collection, Gift of J...",NaN
115,410683,Drawings and Prints,Baseball card,"Louis Chiozza; verso: Diamond Stars, No. 80 (1...",NaN,NaN,1935,1935.0,Commercial color lithograph,"The Jefferson R. Burdick Collection, Gift of J...",NaN
123,433636,Drawings and Prints,"Baseball card, print","Fitzgerald, Sacramento, Pacific Coast League, ...",NaN,NaN,1910,1910.0,Commercial color lithograph,"The Jefferson R. Burdick Collection, Gift of J...",NaN
124,434286,Drawings and Prints,"Baseball card, print","Spencer, Catcher, Boston, American League, fro...",NaN,NaN,1909,1909.0,Photolithograph,"The Jefferson R. Burdick Collection, Gift of J...",NaN
156,634643,Drawings and Prints,"Print, baseball card","Card Number 28, Harry ""The Cat"" Brecheen, Pitc...",NaN,American,1951,1951.0,Commercial color lithograph,"The Jefferson R. Burdick Collection, Gift of J...",NaN
159,642712,Drawings and Prints,"Baseball card, print","Card Number 22, Jim Greengrass, Outfield, Cinc...",NaN,American,1954,1954.0,Commercial color lithograph,"The Jefferson R. Burdick Collection, Gift of J...",NaN
178,705128,Drawings and Prints,"Baseball card, print","Jo-Jo White, from the Goudey Wide Pen Premiums...",NaN,NaN,1936,1936.0,Photolithograph,"The Jefferson R. Burdick Collection, Gift of J...",NaN
179,705189,Drawings and Prints,"Baseball card, print","""Cy"" Blanton, from the National Chicle Fine Pe...",NaN,NaN,1936,1936.0,Photolithograph,"The Jefferson R. Burdick Collection, Gift of J...",NaN
180,708376,Drawings and Prints,"Baseball card, print","Album 324, Page 86",NaN,NaN,1936,1936.0,Commercial Lithographs,"The Jefferson R. Burdick Collection, Gift of J...",NaN
193,751829,Drawings and Prints,"Print, baseball","Silas K. Johnson, Cincinnati Reds, from the Ba...",NaN,NaN,1934,1934.0,Printed matchbook,"The Jefferson R. Burdick Collection, Gift of J...",NaN


In [27]:
for i in req_data[(req_data['Object Name'] == 'Baseball card') | (req_data['Object Name'] == 'Baseball card, print') |
         (req_data['Object Name'] == 'Print, baseball card') | (req_data['Object Name'] == 'Print, baseball')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Object Name'][idx] = 'Baseball card'

<ipython-input-27-4f7d96898999>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Object Name'][idx] = 'Baseball card'


#### CHECKING THE UNIQUE VALUES AFTER STANDARDISING

In [28]:
req_data['Object Name'].unique()

array(['Side Chair', 'Pitcher', 'Plate', 'Bowl', 'Painting, miniature',
       'Arrowhead (Yanonē)', 'Knife handle (Kozuka)', 'Coin (Thaler)',
       'Two drawings', 'Partisan', 'Sword guard (Tsuba)',
       'Dirk with sheath', 'Hanging scroll mounted as handscroll',
       'Girdle ornament', 'Bracelet', 'Print', 'Album leaf', 'Inrō',
       'Basket', 'Piece', 'Hanging scroll', 'Drawing', 'Belt', 'Pumps',
       'Evening dress', 'Shoes', 'Earrings', 'Cap', 'Hat', 'Mules',
       'Gloves', 'Ensemble', 'Necklace', 'Jacket', 'Chemisette', 'Fan',
       'Undersleeves', 'Evening shoes', 'Knife and fork with case',
       'Swags', 'Jug with cover', 'Plaque', 'Snuffbox', 'Chess set',
       'Toy', 'Souvenir spoon', 'Border', 'Fragment', 'Borders', 'Panel',
       'Statuette of a seated goddess', 'Axehead',
       'Fibula, boat-shaped type', 'Statuette of a woman, seated', 'Seal',
       'Photograph', 'Glass positive', 'Negative', 'Horn', 'Snake figure',
       'Pendant', 'Painting', 'Cloth', 

In [29]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             0
Title                  10
Culture               119
Artist Nationality    116
Object Begin Date       0
Object End Date         2
Medium                  2
Credit Line             0
Country               172
dtype: int64

## MISSING VALUES IN THE 'Title' COLUMN

So far, we tried to navigate to the Museum's website and extract information - all of this done manually.
<B>We can see that this task would become tedious as the number of missing entries increases.</B>
  
So, instead of manually going to the website, we use the <B>museum's API to extract missing values in our dataset</B>
  
* <U>Note that we extract only those values that are missing in our dataset.</U>
 
* These missing values may or may not be present in the museum's website.
 
* If they are not present, we make a note of such records and deal with them later

The museum's api can be found here :

<a>https://collectionapi.metmuseum.org/public/collection/v1/objects/[ObjectID]</a>

Replace the [ObjectID] with the object id of the row with the missing value

In [30]:
for i in req_data[req_data['Title'].isnull()]['Object ID']:
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/' + str(i)
    response_API = requests.get(url)
    data = response_API.text
    parse_json = json.loads(data)
    
    req_data['Title'][idx] = parse_json['title']

<ipython-input-30-922c27498abe>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Title'][idx] = parse_json['title']


Having extracted the missing titles, we want to make sure that they are actual text, and not just empty spaces. So we replace empty spaces in the column with 'Nan' values

In [31]:
req_data['Title'] = req_data['Title'].replace(r'^\s*$', np.nan, regex=True)

In [32]:
for i in range(req_data.shape[0]):
    if not req_data['Title'][i] != req_data['Title'][i]:
        req_data['Title'][i] = req_data['Title'][i].strip()

<ipython-input-32-4a510629b8ae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Title'][i] = req_data['Title'][i].strip()


Since each title is unique and cannot be combined together for a smaller number of levels/factors, we persist with the existing titles

In [33]:
req_data['Title'].unique()

array(['Side Chair', 'Covered Pitcher', 'Plate', 'Soup Bowl',
       'Portrait of a Lady', 'Arrowhead (Yanonē)',
       'Knife Handle (Kozuka)',
       'Coin (Thaler) Showing August I (The Pious) of Saxony',
       'Drawings Showing Combat on Foot (Champ Clos)', 'Partisan',
       'Sword Guard (Tsuba)', 'Dirk with Sheath', 'Bamboo',
       'Girdle ornament', 'Open Bracelet',
       'Furuichi Dance (No. 2 of a Set of Four)', 'Cat Catching a Frog',
       'Moonlight View of Suihiro Bridge, Tempozan',
       'Case (Inrō) with Design of Samurai Helmet and Face Mask (obverse); Samurai Arm Guard and Silver Glove (reverse)',
       'Basket', 'Piece', '近代  蟬竹圖      軸|Bamboo and Cicada',
       'Skirmish with an Elephant', 'Belt', 'Pumps', 'Evening dress',
       'Shoes', 'Earrings', 'Cap', 'Hat', 'Mules', 'Gloves', 'Ensemble',
       'Necklace', 'Jacket', 'Chemisette', 'Fan', 'Undersleeves',
       'Evening shoes', 'Knife and fork with case', 'Set of five swags',
       'Jug with cover', 'Marr

#### We notice that the Title column has its missing values filled with proper entries from the Museum's API

In [34]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             0
Title                   0
Culture               119
Artist Nationality    116
Object Begin Date       0
Object End Date         2
Medium                  2
Credit Line             0
Country               172
dtype: int64

## DEALING WITH DISCREPANCIES IN THE 'Culture' COLUMN

Similar to what we did with the <B>'Title'</B> Column, we use the API to fill in the missing values in the <B>'Culture'</B> Column

In [35]:
catch_Culture = []
for i in req_data[req_data['Culture'].isnull()]['Object ID']:
    
    try:
        idx = req_data.index[req_data['Object ID'] == i][0]
        url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/' + str(i)
        response_API = requests.get(url)
        data = response_API.text
        parse_json = json.loads(data)
        
        req_data['Culture'][idx] = parse_json['culture']
    
    except :
        catch_Culture.append(i)
    
    else:
        continue

<ipython-input-35-fb87f86ec5f2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = parse_json['culture']


As stated earlier, all those records that don't have an entry in the museum's webpage are noted and will be dealt with later

In [36]:
catch_Culture

[410683,
 430474,
 632372,
 634643,
 638012,
 642712,
 705128,
 705189,
 708376,
 751829]

#### THE IMPUTED VALUES ARE THEN CHECKED IF THEY CONTAIN EMPTY SPACES INSTEAD OF PROPER VALUES
#### SUCH VALUES ARE REPLACED WITH Nan VALUES
#### THIS OPERATION WILL BE CARRIED OUT FOR ALL COLUMNS THAT ARE IMPUTED USING THE MUSEUM'S API

In [37]:
req_data['Culture'] = req_data['Culture'].replace(r'^\s*$', np.nan, regex=True)

In [38]:
for i in range(req_data.shape[0]):
    if not req_data['Culture'][i] != req_data['Culture'][i]:
        req_data['Culture'][i] = req_data['Culture'][i].strip()

<ipython-input-38-169e8355628a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][i] = req_data['Culture'][i].strip()


Again, we notice that the column values are unstandardised. They also have multiple entries referring to a single entity.

* For example, we have the <B>Japanese</B> and <B>Japan</B> cultures, both denoting the same culture.
* Ideally, we would have one single level that captures one culture uniformly, across all recorded data points

In [39]:
req_data['Culture'].unique()

array(['American', 'Dutch', 'Japanese', 'German', 'possibly Spanish',
       'China', 'Thailand', 'Japan', 'India (Guler, Punjab Hills)',
       'British', 'Italian', 'European, Eastern', 'French',
       'probably French', 'Romanian', 'European', 'possibly Greek',
       'German, Silesia (Bunzlau)', 'Italian, Venice', 'German, Hanau',
       'probably German', 'British, Chelsea', 'Spanish', 'Cypriot',
       'Italic', 'Greek, Attic', 'Minoan', 'Minoan, Crete', nan,
       'Asmat people', 'Aztec', 'Atlantic Watershed', 'Abelam people',
       'Quechua', 'Mexican', 'Timor', 'Iran', 'Byzantine', 'Catalan',
       'South Netherlandish', 'European or Middle Eastern',
       'British, Stoke-on-Trent, Staffordshire'], dtype=object)

So we go ahead and manually change the existing levels in the column to a common level, such that each culture is denoted by one particular level.

Here, 'Japanese' and 'Japan' are replaced with one common term - 'Japanese' 

In [40]:
for i in req_data[(req_data['Culture'] == 'Japanese') | (req_data['Culture'] == 'Japan')]['Object ID']:
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'Japanese'

<ipython-input-40-5485dc501cfa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'Japanese'


In [41]:
req_data['Culture'].unique()

array(['American', 'Dutch', 'Japanese', 'German', 'possibly Spanish',
       'China', 'Thailand', 'India (Guler, Punjab Hills)', 'British',
       'Italian', 'European, Eastern', 'French', 'probably French',
       'Romanian', 'European', 'possibly Greek',
       'German, Silesia (Bunzlau)', 'Italian, Venice', 'German, Hanau',
       'probably German', 'British, Chelsea', 'Spanish', 'Cypriot',
       'Italic', 'Greek, Attic', 'Minoan', 'Minoan, Crete', nan,
       'Asmat people', 'Aztec', 'Atlantic Watershed', 'Abelam people',
       'Quechua', 'Mexican', 'Timor', 'Iran', 'Byzantine', 'Catalan',
       'South Netherlandish', 'European or Middle Eastern',
       'British, Stoke-on-Trent, Staffordshire'], dtype=object)

A similar combining of levels is done for all <B>German</B> cultures

In [42]:
for i in req_data[(req_data['Culture'] == 'German') | (req_data['Culture'] == 'German, Silesia (Bunzlau)') |
         (req_data['Culture'] == 'German, Hanau') | (req_data['Culture'] == 'probably German')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'German'

<ipython-input-42-b4b09a974498>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'German'


Combining levels for <B>Dutch</B> cultures

In [43]:
for i in req_data[(req_data['Culture'] == 'Dutch') | (req_data['Culture'] == 'South Netherlandish')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'Dutch'

<ipython-input-43-72f88cc73eea>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'Dutch'


...And so on

In [44]:
for i in req_data[(req_data['Culture'] == 'possibly Spanish') | (req_data['Culture'] == 'Spanish')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'Spanish'

<ipython-input-44-064d897b7c05>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'Spanish'


In [45]:
for i in req_data[(req_data['Culture'] == 'British') | (req_data['Culture'] == 'British, Chelsea') |
         (req_data['Culture'] == 'British, Stoke-on-Trent, Staffordshire')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'British'


    
    

for i in req_data[(req_data['Culture'] == 'Italian') | (req_data['Culture'] == 'Italic') |
         (req_data['Culture'] == 'Italian, Venice')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'Italian'


    
    
    
    
for i in req_data[(req_data['Culture'] == 'European') | (req_data['Culture'] == 'European, Eastern') |
         (req_data['Culture'] == 'European or Middle Eastern')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'European'

    
    
    

for i in req_data[(req_data['Culture'] == 'French') | (req_data['Culture'] == 'probably French')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'French'
    
    

    

for i in req_data[(req_data['Culture'] == 'possibly Greek') | (req_data['Culture'] == 'Greek, Attic')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'Greek'

    
    
    
    
    
for i in req_data[(req_data['Culture'] == 'Minoan') | (req_data['Culture'] == 'Minoan, Crete')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'Minoan'

<ipython-input-45-b13162b47965>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'British'
<ipython-input-45-b13162b47965>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'Italian'
<ipython-input-45-b13162b47965>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'European'
<ipython-input-45-b13162b47965>:43: SettingWithCopyWarning: 
A value is trying to 

<B>After cleaning the data, the values look much better</B>

In [46]:
req_data['Culture'].unique()

array(['American', 'Dutch', 'Japanese', 'German', 'Spanish', 'China',
       'Thailand', 'India (Guler, Punjab Hills)', 'British', 'Italian',
       'European', 'French', 'Romanian', 'Greek', 'Cypriot', 'Minoan',
       nan, 'Asmat people', 'Aztec', 'Atlantic Watershed',
       'Abelam people', 'Quechua', 'Mexican', 'Timor', 'Iran',
       'Byzantine', 'Catalan'], dtype=object)

#### HOWEVER, THE NUMBER OF MISSING VALUES HASN'T GONE DOWN. THE MUSEUM'S API ALSO HAS MISSING VALUES FOR THESE ENTRIES. 
#### WE DEAL WITH THESE LATER

In [47]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             0
Title                   0
Culture               119
Artist Nationality    116
Object Begin Date       0
Object End Date         2
Medium                  2
Credit Line             0
Country               172
dtype: int64

## DEALING WITH THE COLUMN 'Artist Nationality'

Using the Museum's API, we look to impute missing values with their actual values as recorded in the museum's web page

In [48]:
catch_ArtistNationality = []
for i in req_data[req_data['Artist Nationality'].isnull()]['Object ID']:
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/' + str(i)
    response_API = requests.get(url)
    data = response_API.text
    parse_json = json.loads(data)
    try:
        req_data['Artist Nationality'][idx] = parse_json['artistNationality']
    except:
        catch_ArtistNationality.append(i)
    else:
        continue

<ipython-input-48-78ef10f821eb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Artist Nationality'][idx] = parse_json['artistNationality']


Again, we record the Object IDs of the data points that do not have an associated record in the web-page

In [49]:
catch_ArtistNationality

[410683, 632372, 705128, 705189, 708376, 751829]

replacing any empty spaces that might have been written in from the Museum's API with Nan values, and later trimming any leading or trailing spaces from all entries:

In [50]:
req_data['Artist Nationality'] = req_data['Artist Nationality'].replace(r'^\s*$', np.nan, regex=True)

In [51]:
for i in range(req_data.shape[0]):
    if not req_data['Artist Nationality'][i] != req_data['Artist Nationality'][i]:
        req_data['Artist Nationality'][i] = req_data['Artist Nationality'][i].strip()

<ipython-input-51-9a1c5455ff30>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Artist Nationality'][i] = req_data['Artist Nationality'][i].strip()


Finally, we take a look at the values contained in the column. We see a lot of non standardised values, and repeated inconsistencies.
* So we fix that by modifying each entry - remove duplicates, trailing/leading spaces

In [52]:
req_data['Artist Nationality'].unique()

array([nan, 'Chinese', 'Japanese', 'Japanese|Japanese', 'British',
       'American|American', 'Italian', 'French, born Italy',
       'French|French', 'American', 'British|British', 'Belgian',
       'French', 'French|French|French|French|French|French',
       'American|American|American', 'French|French?|French',
       'British|British|British|British|British|British|British',
       'Italian|Bohemian', 'Dutch|Dutch|Dutch',
       'French|French|French|French|French', 'Dutch',
       'Netherlandish|German', 'Italian|Italian', 'French|British',
       'French|Italian', 'German', 'American, born Russia',
       'Finnish|Finnish', 'British, Scottish', 'Mexican',
       'Italian|Italian|Italian|Italian|Italian',
       'Italian|Italian|German', 'Canadian',
       'Scottish|British|British, Scottish|British|British', 'Austrian',
       'Italian|British', 'German|American|American|American'],
      dtype=object)

In [53]:
for k in req_data['Artist Nationality']:
    if not k != k:
        k_idx = req_data.index[req_data['Artist Nationality'] == k][0]
        k = k.split(', ')
        k = list(set(k))
        for word in k:
            w_idx = k.index(word)
            word = word.split('|')
            for word_n in word:
                word_ne = re.sub('[^A-Za-z, ]+', '', word_n)
                word[word.index(word_n)] = word_ne.strip()
            word = list(set(word))
            word = (', ').join(word)
            k[w_idx] = word
        k = (', ').join(k)
        req_data['Artist Nationality'][k_idx] = k

<ipython-input-53-f0bddf7c4d7e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Artist Nationality'][k_idx] = k


We notice that the data looks much cleaner now

In [54]:
req_data['Artist Nationality'].unique()

array([nan, 'Chinese', 'Japanese', 'British', 'American', 'Italian',
       'French, born Italy', 'French', 'Belgian', 'Italian, Bohemian',
       'Dutch', 'Netherlandish, German', 'French, British',
       'French, Italian', 'German', 'American, born Russia', 'Finnish',
       'Scottish, British', 'Mexican', 'German, Italian', 'Canadian',
       'British, Scottish', 'Austrian', 'Italian, British',
       'German, American'], dtype=object)

#### ONCE AGAIN, WE NOTICE THAT THE NUMBER OF MISSING VALUES IS UNCHANGED

In [55]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             0
Title                   0
Culture               119
Artist Nationality    116
Object Begin Date       0
Object End Date         2
Medium                  2
Credit Line             0
Country               172
dtype: int64

## DEALING WITH THE 'Object Begin Date' AND 'Object End Date' COLUMNS

<I>Object Begin Date </I>does not have any missing values, but the recorded values are a little intriguing, as some of the years are recorded using negative numbers - this is done to indicate the eras (BC, AD)

To avoid confusion, we rectify this by adding a huge value to all dates - ensuring that all recorded dates are positive

In [56]:
min(req_data['Object Begin Date'])

-7000

In [57]:
max(req_data['Object Begin Date'])

2003

We add a value of 100000 to each date to ensure no negative numbers are used to store dates. This can easily be corrected for in later analyses to obtain the proper years

In [58]:
req_data['Object Begin Date'] = req_data['Object Begin Date'].add(100000)

<I>Object End Date</I> has two missing values. We try to impute them using the Museum's API

In [59]:
for i in req_data[req_data['Object End Date'].isnull()]['Object ID']:
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/' + str(i)
    response_API = requests.get(url)
    data = response_API.text
    parse_json = json.loads(data)
    
    req_data['Object End Date'][idx] = parse_json['objectEndDate']

<ipython-input-59-2a62f3e5643a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Object End Date'][idx] = parse_json['objectEndDate']


These dates are also recorded as floating point values (float64 datatype). Since years are all whole numbers, and we want to ensure that the data types of Begin Date and End Date match, we convert the float64 data type into an int64 type

In [60]:
req_data['Object End Date'] = req_data['Object End Date'].astype(np.int64)

Again, some of the dates contain negative integers to represent the BC era. We correct for this the same way we dealt with Begin Dates - by adding a huge number to all dates

In [61]:
min(req_data['Object End Date'])

-5000

In [62]:
max(req_data['Object End Date'])

2004

We add a value of 100000 to each date to ensure no negative numbers are used to store dates. This can easily be corrected for in later analyses to obtain the proper years

In [63]:
req_data['Object End Date'] = req_data['Object End Date'].add(100000)

#### WE NOTICE THAT ALL THE DATE VALUES HAVE BEEN IMPUTED PROPERLY AND CORRECTED TO AVOID NEGATIVE YEARS

In [64]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             0
Title                   0
Culture               119
Artist Nationality    116
Object Begin Date       0
Object End Date         0
Medium                  2
Credit Line             0
Country               172
dtype: int64

## DEALING WITH THE 'Medium' COLUMN

In [65]:
req_data['Medium'].unique()

array(['Mahogany, ebony, ash, birch, white pine', 'Earthenware',
       'Porcelain', 'Watercolor on ivory', 'Steel',
       'Copper-gold alloy (shakudō), copper-silver alloy (shibuichi), gold',
       'Silver', 'Pen, ink and watercolor on paper',
       'Steel, wood, textile, metallic thread',
       'Iron, gold, silver, copper',
       'Steel, horn, brass, leather, copper wire',
       'Hanging scroll mounted as a handscroll; ink on paper', 'Jade',
       'Bronze',
       'Polychrome woodblock print (surimono); ink and color on paper',
       'Album leaf; ink and color on silk',
       'Polychrome woodblock print; ink and color on paper',
       'Lacquer, hirame ground, gold, red, silver and brown hiramakie, raden inlay; Interior: nashiji and fundame',
       'Bamboo', 'Silk / Brocaded',
       'Hanging scroll; ink and color on paper', 'Ink on paper',
       'leather, metal', 'leather', 'rayon', 'synthetic', 'metal',
       '[no medium available]', 'wool', 'wool, cotton',
       'leat

We notice that one of the values contained in the 'Medium' column corresponds to '[no medium available]'. We replace this value with a Nan value

In [66]:
m_idx = req_data.index[req_data['Medium'] == '[no medium available]'][0]

req_data['Medium'][m_idx] = np.nan

<ipython-input-66-612b11de860d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Medium'][m_idx] = np.nan


We use the Museum API to impute the 3 missing values

In [67]:
for i in req_data[req_data['Medium'].isnull()]['Object ID']:
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/' + str(i)
    response_API = requests.get(url)
    data = response_API.text
    parse_json = json.loads(data)
    
    req_data['Medium'][idx] = parse_json['medium']

<ipython-input-67-49e4edd56fa5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Medium'][idx] = parse_json['medium']


Replace any empty strings written in from Museum API with a Nan value

In [68]:
req_data['Medium'] = req_data['Medium'].replace(r'^\s*$', np.nan, regex=True)

In [69]:
m_idx = req_data.index[req_data['Medium'] == '[no medium available]'][0]

req_data['Medium'][m_idx] = np.nan

<ipython-input-69-612b11de860d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Medium'][m_idx] = np.nan


Trim any leading/trailing spaces from each one of the entries

In [70]:
for i in range(req_data.shape[0]):
    if not req_data['Medium'][i] != req_data['Medium'][i]:
        req_data['Medium'][i] = req_data['Medium'][i].strip()

<ipython-input-70-60fad760d9e3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Medium'][i] = req_data['Medium'][i].strip()


<B>Although there are a variety of values included in the 'Medium' column, we do not try to combine them into common values as the medium itself depends heavily on the object it corresponds to. Each object could be vastly different from each other. </B>

* As such, making assumptions about the medium that corresponds to each object, without having looked at the object itself would be very wrong.
* Hence, we do not make any changes to this column's values

In [71]:
req_data['Medium'].unique()

array(['Mahogany, ebony, ash, birch, white pine', 'Earthenware',
       'Porcelain', 'Watercolor on ivory', 'Steel',
       'Copper-gold alloy (shakudō), copper-silver alloy (shibuichi), gold',
       'Silver', 'Pen, ink and watercolor on paper',
       'Steel, wood, textile, metallic thread',
       'Iron, gold, silver, copper',
       'Steel, horn, brass, leather, copper wire',
       'Hanging scroll mounted as a handscroll; ink on paper', 'Jade',
       'Bronze',
       'Polychrome woodblock print (surimono); ink and color on paper',
       'Album leaf; ink and color on silk',
       'Polychrome woodblock print; ink and color on paper',
       'Lacquer, hirame ground, gold, red, silver and brown hiramakie, raden inlay; Interior: nashiji and fundame',
       'Bamboo', 'Silk / Brocaded',
       'Hanging scroll; ink and color on paper', 'Ink on paper',
       'leather, metal', 'leather', 'rayon', 'synthetic', 'metal', nan,
       'wool', 'wool, cotton', 'leather, wood, metal glass',
  

In [72]:
req_data[req_data['Medium'].isnull()]

,Object ID,Department,Object Name,Title,Culture,Artist Nationality,Object Begin Date,Object End Date,Medium,Credit Line,Country
28,96100,Costume Institute,Evening dress,Evening dress,French,"French, born Italy",101966,101967,NaN,"Gift of Mrs. William Rand, 1975",NaN
91,353752,Drawings and Prints,Book,Plates in Missel de Notre-Dame de France,NaN,French,101899,101899,NaN,"The Elisha Whittelsey Collection, The Elisha W...",NaN
169,679516,Photographs,Album,[Album],NaN,NaN,101860,101879,NaN,"Gift of Eva Kasiska, 1947",NaN


#### THE MISSING VALUES IN THE 'Medium' COLUMN STILL REMAIN

In [73]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             0
Title                   0
Culture               119
Artist Nationality    116
Object Begin Date       0
Object End Date         0
Medium                  3
Credit Line             0
Country               172
dtype: int64

## DEALING WITH THE 'Credit Line' COLUMN

This column has no missing values. We however, run the empty space replacement code, and the strip() function on each one of these values to ensure consistency

Again, we have no scope of combining individual levels of the column as each object is credited to a particular individual/institution and it varies from object to object. As such, there can be no common grouping

In [74]:
req_data['Credit Line'].unique()

array(['Friends of the American Wing Fund, 1962', 'Rogers Fund, 1913',
       'Rogers Fund, 1933', 'Gift of Mr. and Mrs. Franklin Chace, 1969',
       'Fletcher Fund, 2006',
       'The Collection of Giovanni P. Morosini, presented by his daughter Giulia, 1932',
       'The Howard Mansfield Collection, Gift of Howard Mansfield, 1936',
       'Gift of Bashford Dean, 1923',
       'Bashford Dean Memorial Collection, Funds from various donors, 1929',
       'Bashford Dean Memorial Collection, Gift of Edward S. Harkness, 1929',
       'H. O. Havemeyer Collection, Bequest of Mrs. H. O. Havemeyer, 1929',
       'Gift of Rutherfurd Stuyvesant, 1891',
       'Bequest of John M. Crawford Jr., 1988',
       'Gift of Samuel T. Peters, 1913',
       'Samuel Eilenberg Collection, Bequest of Samuel Eilenberg, 1998',
       'Charles Stewart Smith Collection, Gift of Mrs. Charles Stewart Smith, Charles Stewart Smith Jr., and Howard Caswell Smith, in memory of Charles Stewart Smith, 1914',
       'Roge

Replacing empty spaces with Nan values

In [75]:
req_data['Credit Line'] = req_data['Credit Line'].replace(r'^\s*$', np.nan, regex=True)

Trimming whitespaces

In [76]:
for i in range(req_data.shape[0]):
    if not req_data['Credit Line'][i] != req_data['Credit Line'][i]:
        req_data['Credit Line'][i] = req_data['Credit Line'][i].strip()

<ipython-input-76-0e87e88022e7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Credit Line'][i] = req_data['Credit Line'][i].strip()


In [77]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             0
Title                   0
Culture               119
Artist Nationality    116
Object Begin Date       0
Object End Date         0
Medium                  3
Credit Line             0
Country               172
dtype: int64

## DEALING WITH 'Country' COLUMN

In [78]:
req_data['Country'].unique()

array(['United States', 'Netherlands', nan, 'Indonesia', 'Mexico',
       'Costa Rica', 'Papua New Guinea', 'Peru', 'Egypt', 'India', 'Iran',
       'present-day Uzbekistan', 'France', 'present-day France'],
      dtype=object)

Again, we make use of the Museum API to impute missing values in this column.

We also track those objects that are not listed in the museum's web page


In [79]:
catch_country = []
for i in req_data[req_data['Country'].isnull()]['Object ID']:
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/' + str(i)
    response_API = requests.get(url)
    data = response_API.text
    parse_json = json.loads(data)
    
    try:
        req_data['Country'][idx] = parse_json['country']
    
    except:
        catch_country.append(i)
    
    else:
        continue

<ipython-input-79-4d5c59558743>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Country'][idx] = parse_json['country']


Replacing any empty spaces written in from the Museum's API with Nan values

In [80]:
req_data['Country'] = req_data['Country'].replace(r'^\s*$', np.nan, regex=True)

Trimming whitespaces

In [81]:
for i in range(req_data.shape[0]):
    if not req_data['Country'][i] != req_data['Country'][i]:
        req_data['Country'][i] = req_data['Country'][i].strip()

<ipython-input-81-cad4221c72ca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Country'][i] = req_data['Country'][i].strip()


#### WE NOTICE THAT SOME OF THE MISSING VALUES IN THIS COLUMN WERE IMPUTED - THE NUMBER DROPPED FROM 172 TO 159

In [82]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             0
Title                   0
Culture               119
Artist Nationality    116
Object Begin Date       0
Object End Date         0
Medium                  3
Credit Line             0
Country               159
dtype: int64

## WE STILL HAVE A SIGNIFICANT NUMBER OF RECORDS WITH MISSING VALUES

With no other way to impute this missing data, we look at the dataset itself to give us some pointers

'Culture' and 'Country' are quite similar to each other - 

* Culture records the culture to which each object belongs to

* Country records the country where the object was found.

It is highly likely that most of the times the country where an object is found happens to be the culture that the object was created in/by.

<B>This is a reasonable argument to :
    * impute missing values in culture with the corresponding country values
    * impute missing values in country with the corresponding culture values

* <B>We notice that a majority of the records (97) don't have any values in both the Culture and Country columns.</B>
* <B>Therefore, the only option is to delete them.</B>

In [83]:
req_data[(req_data['Culture'].isnull()) & (req_data['Country'].isnull())]

,Object ID,Department,Object Name,Title,Culture,Artist Nationality,Object Begin Date,Object End Date,Medium,Credit Line,Country
62,262991,Photographs,Photograph,[Detail of Vanitas Figures Carved on the Choir...,NaN,NaN,101860,101869,Albumen silver print from glass negative,"Gift of Lucien Goldschmidt, 1981",NaN
63,263514,Photographs,Photograph,Robert Motherwell,NaN,American,101981,101981,Gelatin silver print,"Gift of the artist, 1983",NaN
64,265758,Photographs,Photograph,[Graffiti on Backstage Door of the School of A...,NaN,American,101945,101945,Gelatin silver print,"Gift of Paul F. Walter, in memory of Christoph...",NaN
65,266167,Photographs,Photograph,[Study of Australian Grapes],NaN,NaN,101858,101862,Albumen silver print from glass negative,"Purchase, The Horace W. Goldsmith Foundation G...",NaN
66,269735,Photographs,Photograph,Sous bois,NaN,Belgian,101922,101922,Bromoil print,"Gift of Clarence McK. Lewis, 1954",NaN
...,...,...,...,...,...,...,...,...,...,...,...
198,779306,Drawings and Prints,Trade card,"Packard-Tourist, from the ""Antique Autos"" seri...",NaN,NaN,101953,101953,Commercial color lithograph,"The Jefferson R. Burdick Collection, Gift of J...",NaN
199,785013,Drawings and Prints,Print; ephemera,Norwegian Air Force In Canada Patrol Bomber (N...,NaN,NaN,101942,101942,Commercial color lithograph,"The Jefferson R. Burdick Collection, Gift of J...",NaN
200,813925,Drawings and Prints,Print,Stonehenge,NaN,NaN,101979,101979,Sugar-lift aquatint with roulette,"Gift of Jerry S. Flum, 1980",NaN
201,820618,Drawings and Prints,Print,"Psyche, holding the flask of beauty being tran...",NaN,NaN,101525,101535,Engraving,"Bequest of Grace Pugh, 1985",NaN


<B>But first, we impute the culture with country values and country with culture</B>

In [84]:
for i in req_data[req_data['Culture'].isnull()]['Object ID']:
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = req_data['Country'][idx]

    

    

for i in req_data[req_data['Country'].isnull()]['Object ID']:
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Country'][idx] = req_data['Culture'][idx]

<ipython-input-84-ec7d775f6354>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = req_data['Country'][idx]
<ipython-input-84-ec7d775f6354>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Country'][idx] = req_data['Culture'][idx]


<B>Checking if such an imputation introduces any irregularities in the 'CULTURE' Column</B>

In [85]:
req_data['Culture'].unique()

array(['American', 'Dutch', 'Japanese', 'German', 'Spanish', 'China',
       'Thailand', 'India (Guler, Punjab Hills)', 'British', 'Italian',
       'European', 'French', 'Romanian', 'Greek', 'Cypriot', 'Minoan',
       nan, 'Asmat people', 'Aztec', 'Atlantic Watershed',
       'Abelam people', 'Quechua', 'Mexican', 'Timor', 'Iran', 'France',
       'England', 'Russia', 'Egypt', 'India', 'present-day Uzbekistan',
       'Byzantine', 'Catalan'], dtype=object)

<B> Addressing these irregularities </B>

In [86]:
for i in req_data[(req_data['Culture'] == 'India (Guler, Punjab Hills)') | (req_data['Culture'] == 'India')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'India'

    

for i in req_data[(req_data['Culture'] == 'French') | (req_data['Culture'] == 'France')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'French'
    
    
    
for i in req_data[(req_data['Culture'] == 'England') | (req_data['Culture'] == 'British')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Culture'][idx] = 'British'

<ipython-input-86-812e2b0fa65d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'India'
<ipython-input-86-812e2b0fa65d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'French'
<ipython-input-86-812e2b0fa65d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Culture'][idx] = 'British'


<B> The column 'Culture' is clean now </B>

In [87]:
req_data['Culture'].unique()

array(['American', 'Dutch', 'Japanese', 'German', 'Spanish', 'China',
       'Thailand', 'India', 'British', 'Italian', 'European', 'French',
       'Romanian', 'Greek', 'Cypriot', 'Minoan', nan, 'Asmat people',
       'Aztec', 'Atlantic Watershed', 'Abelam people', 'Quechua',
       'Mexican', 'Timor', 'Iran', 'Russia', 'Egypt',
       'present-day Uzbekistan', 'Byzantine', 'Catalan'], dtype=object)

<B>Checking if such an imputation introduces any irregularities in the 'COUNTRY' Column</B>

In [88]:
req_data['Country'].unique()

array(['United States', 'Netherlands', 'American', 'Japan', 'Germany',
       'possibly Spain', 'China', 'Thailand', 'Japanese',
       'India (Guler, Punjab Hills)', 'British', 'Italian', 'European',
       'French', nan, 'Romanian', 'Greek', 'German', 'Spanish', 'Cypriot',
       'Minoan', 'Indonesia', 'Mexico', 'Costa Rica', 'Papua New Guinea',
       'Peru', 'Iran', 'France', 'England', 'Russia', 'Egypt', 'India',
       'present-day Uzbekistan', 'Byzantine', 'present-day France',
       'Dutch'], dtype=object)

<B> Addressing these irregularities </B>

In [89]:
for i in req_data[(req_data['Country'] == 'India (Guler, Punjab Hills)') | (req_data['Country'] == 'India')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Country'][idx] = 'India'

    

for i in req_data[(req_data['Country'] == 'French') | (req_data['Country'] == 'France') | 
                 (req_data['Country'] == 'present-day France')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Country'][idx] = 'France'
    
    
    
for i in req_data[(req_data['Country'] == 'England') | (req_data['Country'] == 'British')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Country'][idx] = 'British'
    
    
    
    
for i in req_data[(req_data['Country'] == 'Japan') | (req_data['Country'] == 'Japanese')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Country'][idx] = 'Japan'
    
    
    
    
    
for i in req_data[(req_data['Country'] == 'United States') | (req_data['Country'] == 'American')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Country'][idx] = 'United States'
    
    
    
    
    
    
for i in req_data[(req_data['Country'] == 'Netherlands') | (req_data['Country'] == 'Dutch')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Country'][idx] = 'Dutch'
    
    
    
    
    
for i in req_data[(req_data['Country'] == 'Germany') | (req_data['Country'] == 'German')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Country'][idx] = 'Germany'
    
    
    
    

for i in req_data[(req_data['Country'] == 'Spanish') | (req_data['Country'] == 'possibly Spain')]['Object ID']:
    
    
    idx = req_data.index[req_data['Object ID'] == i][0]
    
    req_data['Country'][idx] = 'Spain'

<ipython-input-89-dd3ea75dff09>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Country'][idx] = 'India'
<ipython-input-89-dd3ea75dff09>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Country'][idx] = 'France'
<ipython-input-89-dd3ea75dff09>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data['Country'][idx] = 'British'
<ipython-input-89-dd3ea75dff09>:35: SettingWithCopyWarning: 
A value is trying to be s

<B> The column COUNTRY is clean now.</B>

In [90]:
req_data['Country'].unique()

array(['United States', 'Dutch', 'Japan', 'Germany', 'Spain', 'China',
       'Thailand', 'India', 'British', 'Italian', 'European', 'France',
       nan, 'Romanian', 'Greek', 'Cypriot', 'Minoan', 'Indonesia',
       'Mexico', 'Costa Rica', 'Papua New Guinea', 'Peru', 'Iran',
       'Russia', 'Egypt', 'present-day Uzbekistan', 'Byzantine'],
      dtype=object)

## DESPITE SUCH AN IMPUTATION, WE STILL HAVE MISSING VALUES

In [91]:
req_data.isna().sum()

Object ID               0
Department              0
Object Name             0
Title                   0
Culture                98
Artist Nationality    116
Object Begin Date       0
Object End Date         0
Medium                  3
Credit Line             0
Country                98
dtype: int64

## The column 'Artist Nationality' cannot be imputed in a similar fashion.

This is because there is a <B>significant variability associated with the person who makes the finding and the place of the finding</B>

Imputing an Artist's nationality based on the culture/country of the finding could wildly corrupt the data.

Hence, <B>The only option is to delete all the records that still have missing values</B>

<B>But first, we make a note of all the records that will be deleted - to ensure that we have a log of all the datapoints removed

In [92]:
removed_rows = list(req_data[req_data.isnull().any(axis = 1)].index)

In [93]:
removed_rows

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 18,
 19,
 20,
 22,
 27,
 28,
 29,
 31,
 33,
 34,
 36,
 38,
 39,
 40,
 43,
 44,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 87,
 89,
 90,
 91,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202]

<B> Finally, we remove all rows with Nan values, and only keep fully valid records

In [94]:
finale = req_data.drop(removed_rows)

In [95]:
finale

,Object ID,Department,Object Name,Title,Culture,Artist Nationality,Object Begin Date,Object End Date,Medium,Credit Line,Country
12,39917,Asian Art,Hanging scroll mounted as handscroll,Bamboo,China,Chinese,101037,101101,Hanging scroll mounted as a handscroll; ink on...,"Bequest of John M. Crawford Jr., 1988",China
15,54547,Asian Art,Print,Furuichi Dance (No. 2 of a Set of Four),Japanese,Japanese,101800,101899,Polychrome woodblock print (surimono); ink and...,"H. O. Havemeyer Collection, Bequest of Mrs. H....",Japan
16,54626,Asian Art,Album leaf,Cat Catching a Frog,Japanese,Japanese,101877,101897,Album leaf; ink and color on silk,"Charles Stewart Smith Collection, Gift of Mrs....",Japan
17,55442,Asian Art,Print,"Moonlight View of Suihiro Bridge, Tempozan",Japanese,Japanese,101838,101838,Polychrome woodblock print; ink and color on p...,"Rogers Fund, 1922",Japan
21,72803,Asian Art,Hanging scroll,近代 蟬竹圖 軸|Bamboo and Cicada,China,Chinese,101966,101966,Hanging scroll; ink and color on paper,"The Lin Yutang Family Collection, Gift of Rich...",China
23,80735,Costume Institute,Belt,Belt,Japanese,Japanese,101983,101987,"leather, metal","Gift of Linda A. Rosenthal, 1993",Japan
24,81291,Costume Institute,Pumps,Pumps,British,British,101980,101989,leather,"Gift of Holly Solomon, 1999",British
25,81451,Costume Institute,Evening dress,Evening dress,American,American,101983,101984,rayon,"Gift of Calvin Klein, Inc., 1998",United States
26,83780,Costume Institute,Shoes,Shoes,Italian,Italian,101983,101989,synthetic,"Gift of Muriel Kallis Newman, 2002",Italian
30,100359,Costume Institute,Hat,Hat,French,French,101949,101949,"wool, cotton","Gift of Mary S. Ryan, 1981",France


In [96]:
finale.to_csv('clean_data.csv', index = False)

In [98]:
finale.shape[0]

22

In [99]:
finale['Country'].unique()

array(['China', 'Japan', 'British', 'United States', 'Italian', 'France',
       'Russia'], dtype=object)